# Signal Chain Reconstruction

In [2]:
import torch
from torch import nn
import torchaudio
import os
import numpy as np
import matplotlib.pyplot as plt

### Load dataset

In [3]:
from src.gtfxdataset import GtFxDataset

AUDIO_DIR = "_assets/DATASET/GT-FX-C53/"
ANNOTATIONS_FILE = os.path.join(AUDIO_DIR, "annotation.csv")

SAMPLE_RATE = 22050
NUM_SAMPLES = 22050*3

EFFECT_MAP = ["distortion", "chorus", "tremolo", "delay", "reverb"]


if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using device {device}")

mel_spectrogram = torchaudio.transforms.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_fft=512,
    hop_length=1050,
    n_mels=64,
    # power=2
)

spectrogram = torchaudio.transforms.Spectrogram(
    power=2,
    n_fft=127,
    win_length= 127,
    hop_length= 1040,
    normalized=True
)

mfcc = torchaudio.transforms.MFCC(
    sample_rate = SAMPLE_RATE, 
    n_mfcc = 64,
    melkwargs = {
        "n_fft": 1024,
        "hop_length": 1030,
        "n_mels": 64,
        "center": False})

fxData = GtFxDataset(ANNOTATIONS_FILE,
                        AUDIO_DIR,
                        mfcc,
                        SAMPLE_RATE,
                        NUM_SAMPLES,
                        device)
    

Using device cpu


#### Reconstruct Singal Chain

In [5]:
from src.classifier import model as cModel
from src.classifier import train as cTrain
from src.extrector import model as eModel
from src.extrector import train as eTrain
from src.effectapplier import EffectApplier
from src.util import play_audio
import pandas as pd
import random

_WEIGHTS_DIR = "_weights/"
_DRY_SAMPLE_DIR = "_assets/DATASET/test3"

CLASSIFY_WEIGHTS = os.path.join(_WEIGHTS_DIR, "c53_classify.pth")

index = random.randint(0, len(fxData))
test_data = fxData[index]

sample_file = fxData.get_audio_sample_filename(index) + ".wav"
dry_file = sample_file[:2] + "0" + sample_file[3:sample_file.find('_', 4)] + ".wav"

classify = cModel.Classifier().to(device)
classify.load_state_dict(torch.load(CLASSIFY_WEIGHTS))

log = cTrain.test_single(classify, test_data)

extract = eModel.Extractor().to(device)
eff = EffectApplier(_DRY_SAMPLE_DIR, dry_file)

table = []
para_list = ['Gain', 'Depth', 'Rate', 'Delay Time', 'Decay']
for fx, state in enumerate(log):
    table.append([EFFECT_MAP[fx],
                 "State",
                 "Activate" if test_data[1][fx] > 0.0 else "Bypass",
                 "Activate" if state == 1 else "Bypass"])

    if state == 1:
        WEIGHTS_FILE = os.path.join(_WEIGHTS_DIR, "c55_parameter_" + str(fx) + ".pth")
        extract.load_state_dict(torch.load(WEIGHTS_FILE))
        val = eTrain.test_single(extract, test_data, fx)
        eff.addEffect(fx, val, mode="manual")
    else:
        val = -1.0

    table.append([EFFECT_MAP[fx],
                para_list[fx],
                "N/A" if test_data[1][fx] < 1 else round(test_data[3][fx].item(), 2),
                "N/A" if val < 0 else val])

waveform, sr = torchaudio.load(os.path.join(AUDIO_DIR, sample_file))

print(f"Test file: {sample_file}")
play_audio(waveform, sr)

print(f"Remix by AI Model:")
play_audio(eff.generate(), sr)

df = pd.DataFrame(table)
df.columns = ["Effect", "Item", "Expected", "Predicted"]
df2 = df.groupby(['Effect', "Item"])
df2.first()



Test file: c53_14_od_0_cs_2_tr_1.wav


Remix by AI Model:


Expected Predicted
Effect     Item                          
chorus     Depth           0.78      0.65
           State       Activate  Activate
delay      Delay Time       N/A       N/A
           State         Bypass    Bypass
distortion Gain            0.18      0.19
           State       Activate  Activate
reverb     Decay            N/A       N/A
           State         Bypass    Bypass
tremolo    Rate            0.52      0.65
           State       Activate  Activate